In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import warnings
warnings.filterwarnings('ignore')
print("✅ All imports successful!")

✅ All imports successful!


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import warnings
warnings.filterwarnings('ignore')
print("✅ All imports successful!")

✅ All imports successful!


In [3]:
print("Loading dataset...")
df = pd.read_csv('../archive (2)/train.csv')  # Or train (1).csv
print(f"Dataset shape: {df.shape}")
print(f"\nFirst 5 rows:")
df.head()

Loading dataset...
Dataset shape: (1460, 81)

First 5 rows:


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
selected_features = ['OverallQual', 'GrLivArea', 'TotalBsmtSF', 'GarageCars', 'YearBuilt', 'Neighborhood']
target = 'SalePrice'

df_work = df[selected_features + [target]].copy()
print(f"Working dataset shape: {df_work.shape}")
print(f"\nMissing values before handling:")
print(df_work.isnull().sum())

Working dataset shape: (1460, 7)

Missing values before handling:
OverallQual     0
GrLivArea       0
TotalBsmtSF     0
GarageCars      0
YearBuilt       0
Neighborhood    0
SalePrice       0
dtype: int64


In [5]:
print("Handling missing values...")

# Fill numerical features with median
for feature in ['TotalBsmtSF', 'GarageCars']:
    if df_work[feature].isnull().sum() > 0:
        df_work[feature].fillna(df_work[feature].median(), inplace=True)

# Fill categorical features with mode
if df_work['Neighborhood'].isnull().sum() > 0:
    df_work['Neighborhood'].fillna(df_work['Neighborhood'].mode()[0], inplace=True)

print(f"\nMissing values after handling:")
print(df_work.isnull().sum())


Handling missing values...

Missing values after handling:
OverallQual     0
GrLivArea       0
TotalBsmtSF     0
GarageCars      0
YearBuilt       0
Neighborhood    0
SalePrice       0
dtype: int64


In [6]:
print("Encoding categorical variables...")
le = LabelEncoder()
df_work['Neighborhood_Encoded'] = le.fit_transform(df_work['Neighborhood'])

# Save label encoder
joblib.dump(le, 'label_encoder.pkl')
print("✅ Label encoder saved!")
print(f"\nNumber of unique neighborhoods: {len(le.classes_)}")

Encoding categorical variables...
✅ Label encoder saved!

Number of unique neighborhoods: 25


In [7]:
X = df_work[['OverallQual', 'GrLivArea', 'TotalBsmtSF', 'GarageCars', 'YearBuilt', 'Neighborhood_Encoded']]
y = df_work[target]

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"\n✅ Training set: {X_train.shape[0]} samples")
print(f"✅ Testing set: {X_test.shape[0]} samples")

Features shape: (1460, 6)
Target shape: (1460,)

✅ Training set: 1168 samples
✅ Testing set: 292 samples


In [8]:
print("Applying feature scaling...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save scaler
joblib.dump(scaler, 'scaler.pkl')
print("✅ Scaler saved!")

Applying feature scaling...
✅ Scaler saved!


In [9]:
print("="*60)
print("Training Random Forest Regressor...")
print("="*60)

model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train_scaled, y_train)
print("✅ Model training completed!")

Training Random Forest Regressor...
✅ Model training completed!


In [10]:
print("\n" + "="*60)
print("MODEL EVALUATION")
print("="*60)

# Make predictions
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

# Calculate metrics
train_mae = mean_absolute_error(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)

test_mae = mean_absolute_error(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_r2 = r2_score(y_test, y_test_pred)

print("\n📊 TRAINING SET PERFORMANCE:")
print(f"   MAE:  ${train_mae:,.2f}")
print(f"   RMSE: ${train_rmse:,.2f}")
print(f"   R²:   {train_r2:.4f}")

print("\n📊 TESTING SET PERFORMANCE:")
print(f"   MAE:  ${test_mae:,.2f}")
print(f"   RMSE: ${test_rmse:,.2f}")
print(f"   R²:   {test_r2:.4f}")


MODEL EVALUATION

📊 TRAINING SET PERFORMANCE:
   MAE:  $10,440.12
   RMSE: $17,029.24
   R²:   0.9514

📊 TESTING SET PERFORMANCE:
   MAE:  $18,730.77
   RMSE: $29,705.85
   R²:   0.8850


In [11]:
feature_importance = pd.DataFrame({
    'Feature': ['OverallQual', 'GrLivArea', 'TotalBsmtSF', 'GarageCars', 'YearBuilt', 'Neighborhood'],
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\n📈 FEATURE IMPORTANCE:")
print(feature_importance)


📈 FEATURE IMPORTANCE:
        Feature  Importance
0   OverallQual    0.602355
1     GrLivArea    0.190243
2   TotalBsmtSF    0.097746
4     YearBuilt    0.052213
5  Neighborhood    0.029399
3    GarageCars    0.028045


In [12]:
print("\n" + "="*60)
print("SAVING MODEL")
print("="*60)

joblib.dump(model, 'house_price_model.pkl')
print("✅ Model saved as 'house_price_model.pkl'")
print("\n📁 Files created in model/ folder:")
print("   - house_price_model.pkl")
print("   - scaler.pkl")
print("   - label_encoder.pkl")


SAVING MODEL
✅ Model saved as 'house_price_model.pkl'

📁 Files created in model/ folder:
   - house_price_model.pkl
   - scaler.pkl
   - label_encoder.pkl


In [13]:
print("\n🔄 Testing model reload...")

# Load saved files
loaded_model = joblib.load('house_price_model.pkl')
loaded_scaler = joblib.load('scaler.pkl')
loaded_le = joblib.load('label_encoder.pkl')

# Test prediction
sample_data = X_test.iloc[0:1]
sample_scaled = loaded_scaler.transform(sample_data)
prediction = loaded_model.predict(sample_scaled)

print(f"\n✅ Sample Prediction Test:")
print(f"   Predicted: ${prediction[0]:,.2f}")
print(f"   Actual:    ${y_test.iloc[0]:,.2f}")
print(f"   Difference: ${abs(prediction[0] - y_test.iloc[0]):,.2f}")
print("\n🎉 Model successfully saved and can be reloaded!")


🔄 Testing model reload...

✅ Sample Prediction Test:
   Predicted: $138,703.12
   Actual:    $154,500.00
   Difference: $15,796.88

🎉 Model successfully saved and can be reloaded!
